In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Package

In [ ]:
# Basic
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
tqdm.pandas()

# Geopandas 관련 버전 에러 방지
os.environ['USE_PYGEOS'] = '0'

# Load the shp file
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# Load for convert lat/long to geometry
from geopy.geocoders import Nominatim
import urllib
from urllib import parse
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import requests
import json
import time

In [ ]:
# Load for regression
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Download libraries
!pip install libpysal
!pip install esda
!pip install splot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.8/331.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 24.8 MB/s eta 0:00:00


In [ ]:
# Load for Spatial Autocorrelation
import libpysal as pls
import esda
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local

In [ ]:
# Load for Visualization
import folium

# Data Load

In [ ]:
os.chdir('/content/drive/MyDrive/2. 데이터 분석(Python)/Data')

In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1UInkOZo4OULeCnCY3IAEiSVNqWiundgF/2. 데이터 분석(Python)/Data'

In [ ]:
path = "./"
os.listdir(path)

['서울시 상권분석서비스(점포-행정동)_2020년.csv',
 '서울시 상권분석서비스(점포-행정동)_2021년.csv',
 '서울시 상권분석서비스(점포-행정동)_2022년.csv',
 '소상공인시장진흥공단_상가(상권)정보_서울_202312.csv',
 '서울시 상권분석서비스(직장인구-행정동).csv',
 '서울시 상권분석서비스(상주인구-행정동).csv',
 '서울시 상권분석서비스(집객시설-행정동).csv',
 '서울시 광진구 안심이 CCTV 연계 현황.csv',
 '서울시 버스정류소 위치정보.csv',
 '서울교통공사 지하철역 주소 및 전화번호 정보.csv',
 '경찰청_경찰관서 위치 주소 현황_20230811.csv',
 '서울시 자치구별 도보 네트워크 공간정보.csv',
 '서울시 상권분석서비스(점포-행정동)_2023년.csv',
 '서울시 상권분석서비스(영역-행정동)',
 '서울시 상권분석서비스(영역-상권)',
 'processed_data.pkl',
 '서울_공시지가_shp',
 '광진구_공시지가_shp',
 '서울시 병의원 위치 정보.csv',
 '서울시 약국 인허가 정보.csv',
 '광진구 2030 ᄉ

In [ ]:
csv_dict = {} # shp 파일을 제외한 모든 csv 파일은 해당 딕녀너리에 저장, 한글 key값 에러로 숫자로 대체
n = 0

for file in os.listdir(path):
    if file.endswith(".csv"):
        csv_dict[n] = pd.read_csv(path + file, encoding = "UTF-8", encoding_errors="ignore")
        n += 1

<ipython-input-18-344df0c339d0>:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_dict[n] = pd.read_csv(path + file, encoding = "UTF-8", encoding_errors="ignore")
<ipython-input-18-344df0c339d0>:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_dict[n] = pd.read_csv(path + file, encoding = "UTF-8", encoding_errors="ignore")


# Data Pre-Processing

## 행정동 면적대비 직장인구 시각화

In [ ]:
work_popul = csv_dict[16].copy()
work_popul = work_popul[["행정동_코드_명", "행정동_면적대비_직장_인구"]]
work_popul.columns = ["name", "value"]
work_popul

,name,value
0,자양3동,10.41
1,자양4동,9.95
2,구의1동,29.72
3,구의2동,7.83
4,구의3동,43.68
5,화양동,36.94
6,군자동,38.86
7,중곡1동,19.92
8,중곡2동,24.71
9,중곡3동,13.67


In [ ]:
new_code = {
    '자양3동': '1105066',
    '자양4동': '1105067',
    '구의1동': '1105060',
    '구의2동': '1105061',
    '구의3동': '1105062',
    '화양동': '1105053',
    '군자동': '1105054',
    '중곡1동': '1105055',
    '중곡2동': '1105056',
    '중곡3동': '1105057',
    '중곡4동': '1105058',
    '능동': '1105059',
    '광장동': '1105063',
    '자양1동': '1105064',
    '자양2동': '1105065'
}

work_popul["name"] = work_popul["name"].replace(new_code)
work_popul

,name,value
0,1105066,10.41
1,1105067,9.95
2,1105060,29.72
3,1105061,7.83
4,1105062,43.68
5,1105053,36.94
6,1105054,38.86
7,1105055,19.92
8,1105056,24.71
9,1105057,13.67


In [ ]:
# 전국 행정동 geodata에서 광진구만 분류
with open('skorea-submunicipalities-2018-geo.json') as f:
  korea_geo = json.load(f)

gwangjin_codes = list(work_popul["name"])
gwangjin_features = [feature for feature in korea_geo["features"] if feature["properties"]["code"] in gwangjin_codes]

gwangjin_geo = {
    "type": "FeatureCollection",
    "features": gwangjin_features
}

gwangjin_geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[127.07045515667903, 37.5479949074195],
       [127.074127483685, 37.54732288780035],
       [127.07705921066507, 37.54592213665403],
       [127.07731281134923, 37.54583511050422],
       [127.07743261151238, 37.54580333573506],
       [127.07773997265159, 37.545726947725186],
       [127.07954571347089, 37.54535103898045],
       [127.07962971558332, 37.545334080745604],
       [127.07993332745158, 37.54530073221783],
       [127.08155069645214, 37.54530204827631],
       [127.0812426832015, 37.542353463032015],
       [127.08126772576712, 37.54216264326774],
       [127.08127914379108, 37.54213374128002],
       [127.08144476827715, 37.54203850731832],
       [127.08246881445957, 37.5414917189786],
       [127.08297964179627, 37.541358841038154],
       [127.08278032713909, 37.54089413518793],
       [127.08270858334048, 37.540730276879685],
       [127.08242

In [ ]:
with open("gwangjin_geo.json", "w") as f:
  json.dump(gwangjin_geo, f, indent=4)

In [ ]:
# Base Map
population_map = folium.Map(
    location = [37.541, 126.986],
    zoom_start = 12,
    tiles = "openstreetmap"
)

# Add GeoJson
folium.Choropleth(
    geo_data = gwangjin_geo,
    name = "choropleth",
    data = work_popul,
    columns = ["name", "value"],
    key_on = "feature.properties.code",
    fill_color = "Blues",
    fill_opacity = 0.7,
    line_opacity = 0.2
).add_to(population_map)

population_map

## 서울시 광진구 안심이 CCTV 연계 현황.csv

In [ ]:
cctv_relm = csv_dict[17].copy()
cctv_relm = cctv_relm[["행정동_코드_명", "행정동_면적대비_cctv_수"]]
cctv_relm.columns = ["name", "value"]
cctv_relm

,name,value
0,자양3동,1.81
1,자양4동,2.39
2,구의1동,5.06
3,구의2동,2.28
4,구의3동,2.70
5,화양동,2.62
6,군자동,4.21
7,중곡1동,2.95
8,중곡2동,4.34
9,중곡3동,5.17


In [ ]:
cctv_relm["name"] = cctv_relm["name"].replace(new_code)
cctv_relm

,name,value
0,1105066,1.81
1,1105067,2.39
2,1105060,5.06
3,1105061,2.28
4,1105062,2.70
5,1105053,2.62
6,1105054,4.21
7,1105055,2.95
8,1105056,4.34
9,1105057,5.17


In [ ]:
# Base Map
cctv_map = folium.Map(
    location = [37.541, 126.986],
    zoom_start = 12,
    tiles = "openstreetmap"
)

# Add GeoJson
folium.Choropleth(
    geo_data = gwangjin_geo,
    name = "choropleth",
    data = work_popul,
    columns = ["name", "value"],
    key_on = "feature.properties.code",
    fill_color = "Blues",
    fill_opacity = 0.7,
    line_opacity = 0.2
).add_to(cctv_map)

cctv_map